In [11]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline
from sklearn.externals import joblib

In [12]:
df = pd.read_pickle("../dataset.pkl")

In [16]:
x = df.iloc[:,:483]
x.columns
#483 = 3(adm_days,gender,age)+480(lab values)

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483)

In [17]:
y = df.iloc[:,483:]


In [18]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [19]:
from sklearn.model_selection import train_test_split
#train,test,val 
seed = 40
x_tra,x_test,y_tra,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size = 0.2,random_state=seed)

In [20]:
y.shape

(58976, 20)

In [21]:
x_train.shape

(40103, 483)

In [22]:
y_train.shape

(40103, 20)

In [23]:
x_test.shape

(8847, 483)

In [14]:
y_test.shape

(8847, 20)

In [15]:
np.unique(y_train)

array([0, 1])

In [26]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
sample_weights = [1-3032/58976,1-8924/58976,1-35608/58976,1-19366/58976,
                 1-16271/58976,1-20008/58976,1-43243/58976,
                 1-25281/58976,1-21200/58976,1-21782/58976,1-166/58976,          
                 1-8316/58976,1-10053/58976,1-3205/58976,
                 1-5492/58976,1-19454/58976,1-1757/58976,1-23352/58976,1-16621/58976,1-34350/58976]
weight_rf = [{0:3032/58976,1:1-3032/58976},{0:8924/58976,1:1-8924/58976},{0:1-35608/58976,1:35608/58976},{0:19366/58976,1:1-19366/58976},
            {0:16271/58976,1:1-16271/58976},{0:20008/58976,1:1-20008/58976},{0:1-43243/58976,1:43243/58976},
            {0:25281/58976,1:1-25281/58976},{0:21200/58976,1:1-21200/58976},{0:21782/58976,1:1-21782/58976},{0:166/58976,1:1-166/58976},
            {0:8316/58976,1:1-8316/58976},{0:10053/58976,1:1-10053/58976},{0:3205/58976,1:1-3205/58976},
            {0:5492/58976,1:1-5492/58976},{0:19454/58976,1:1-19454/58976},{0:1757/58976,1:1-1757/58976},
            {0:23352/58976,1:1-23352/58976},{0:16621/58976,1:1-16621/58976},{0:1-34350/58976,1:34350/58976}]*58976
weights = {0:0.286,1:0.714}#this for catboost
#[0,1] classes 0.286 is probabbility of 1 in dataset(icd) so i flipped it over as to give more weight to 1

#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1500,learning_rate=0.15,n_jobs=-1,class_weight=weights))
# what are u trying to assign 1500 to?try

In [34]:
'''parameters = {
    
    
    "learning_rate":[0.1, 0.2, 0.3, 0.4]
}

model_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,
                             scoring = 'roc_auc')

model_tunning.fit(x_train, y_train)

print(model_tunning.best_score_)
print(model_tunning.best_params_)'''

'parameters = {\n    \n    \n    "learning_rate":[0.1, 0.2, 0.3, 0.4]\n}\n\nmodel_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,\n                             scoring = \'roc_auc\')\n\nmodel_tunning.fit(x_train, y_train)\n\nprint(model_tunning.best_score_)\nprint(model_tunning.best_params_)'

In [25]:
#estimator.get_params().keys()

In [27]:
cbc = clf_multilabel.fit(x_train,y_train)
joblib.dump(cbc, "LGBM_40.pkl")

['LGBM_40.pkl']

In [18]:
#print(clf_multilabel.best_score_)
mod = joblib.load("LGBM_40.pkl")

In [19]:
y_pred = mod.predict(x_test)

In [20]:
y_pred

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]])

In [21]:
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))
#oooooooooooooooooooooooooooooooo ya

0.5337471312330482
0.4262707262996866
0.6876947014650964
0.20207980106250706


In [ ]:
#y_predtest_prob.to_pickle("./y_predtest_probcat.pkl")

In [ ]:
#y_predtrain_prob.to_pickle("./y_predtrain_probcat.pkl")